<a href="https://colab.research.google.com/github/jinok-cha/Python-for-Finance-Cookbook/blob/master/%ED%80%80%ED%8A%B8%EC%A0%84%EB%9E%B5_%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C_%EC%84%B8%EC%9B%8C%EB%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
from urllib.request import urlopen

In [66]:
def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)

    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    return fs_df

In [ ]:
def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code:dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col] * len(dataframe), temp_df.columns]

        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)
    return total_df

In [83]:
def make_invest_dataframe(firm_code):
  invest_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=701&gicode=' + firm_code
  invest_page = requests.get(invest_url)
  invest_tables = pd.read_html(invest_page.text)
  temp_df = invest_tables[1]

  temp_df = temp_df.set_index(temp_df.columns[0])
  temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                         'PCR계산에 참여한 계정 펼치기',
                         'PSR계산에 참여한 계정 펼치기',
                         'PBR계산에 참여한 계정 펼치기',
                         '총현금흐름']]
  temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
  return temp_df

In [72]:
firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']

for num, code in enumerate(firmcode_list):
    fs_df = make_fs_dataframe(code)
    fs_df_changed = change_df(code, fs_df)
    if num == 0:
        total_fr = fs_df_changed
    else:
        total_fr = pd.concat([total_fr, fs_df_changed])

print(total_fr)

            2019/12                                                       \
IFRS(연결)        매출액      영업이익     당기순이익         자산         부채         자본   
A005930   2304009.0  277685.0  217389.0  3525645.0   896841.0  2628804.0   
A005380   1057464.0   36055.0   31856.0  1945122.0  1181465.0   763658.0   
A035420     43562.0   11550.0    3968.0   122995.0    57956.0    65039.0   
A003550     65345.0    9943.0   11068.0   224346.0    34136.0   190210.0   
A034730    978115.0   37359.0   16060.0  1326114.0   804354.0   521760.0   

                         2020/12                      ...    2021/12  \
IFRS(연결) 영업활동으로인한현금흐름        매출액      영업이익     당기순이익  ...         부채   
A005930      453829.0  2368070.0  359939.0  264078.0  ...  1217212.0   
A005380        4198.0  1039976.0   23947.0   19246.0  ...  1513306.0   
A035420       13568.0    53041.0   12153.0    8450.0  ...    96636.0   
A003550        8187.0    51991.0   15861.0   15407.0  ...    37296.0   
A034730       80277.0   808188.0   

In [84]:
for num, code in enumerate(firmcode_list):
    invest_df = make_invest_dataframe(code)
    invest_df_changed = change_df(code, invest_df)
    if num == 0:
        total_invest = invest_df_changed
    else:
        total_invest = pd.concat([total_invest, invest_df_changed])

print(total_invest)

        2018/12                            2019/12                             \
            PER    PCR   PSR   PBR   총현금흐름     PER    PCR   PSR   PBR   총현금흐름   
A005930    6.42   4.01  1.16  1.10  691786   17.63   7.42  1.65  1.49  494337   
A005380   22.14   6.34  0.34  0.47   53366   11.20   4.90  0.32  0.47   65904   
A035420   30.99  22.10  3.60  3.08    7892   52.72  28.43  7.06  4.27   12624   
A003550    6.23   5.69  1.50  0.65   20476   11.35   9.76  1.88  0.65    9721   
A034730    8.18   2.27  0.18  1.05   93578   25.92   2.23  0.19  1.00  100090   

         ... 2021/12                           2022/03                          
         ...     PER   PCR   PSR   PBR   총현금흐름     PER  PCR  PSR   PBR   총현금흐름  
A005930  ...   13.55  7.24  1.90  1.80  728698     NaN  NaN  NaN  1.54  203914  
A005380  ...   11.71  6.09  0.49  0.75   93329     NaN  NaN  NaN  0.64   27012  
A035420  ...    3.77  3.67  9.12  2.52   13546     NaN  NaN  NaN  2.32    1780  
A003550  ...    5.22  4.89 